# Reading in data

In [3]:
using Gurobi, StatsBase, CSV, DataFrames, JuMP, LinearAlgebra, Distributions, Random, GLMNet, Printf

[ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
[ Info: Precompiling StatsBase [2913bbd2-ae8a-5f71-8c99-4fb6c76f3a91]
[ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
[ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
[ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
[ Info: Precompiling GLMNet [8d5ece8b-de18-5317-b113-243142960cc6]


In [47]:
data = CSV.read("final_df.csv", DataFrame);

In [48]:
first(data, 5)

Row,food,emissions,price,serving_size,calories,total_fat,saturated_fat,cholesterol,sodium,choline,folate,folic_acid,niacin,pantothenic_acid,riboflavin,thiamin,vitamin_a,vitamin_a_rae,carotene_alpha,carotene_beta,cryptoxanthin_beta,lutein_zeaxanthin,lucopene,vitamin_b12,vitamin_b6,vitamin_c,vitamin_d,vitamin_e,tocopherol_alpha,vitamin_k,calcium,copper,irom,magnesium,manganese,phosphorous,potassium,selenium,zink,protein,alanine,arginine,aspartic_acid,cystine,glutamic_acid,glycine,histidine,hydroxyproline,isoleucine,leucine,lysine,methionine,phenylalanine,proline,serine,threonine,tryptophan,tyrosine,valine,carbohydrate,fiber,sugars,fructose,galactose,glucose,lactose,maltose,sucrose,fat,saturated_fatty_acids,monounsaturated_fatty_acids,polyunsaturated_fatty_acids,fatty_acids_total_trans,alcohol,ash,caffeine,theobromine,water
,String31,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Wheat & Rye (Bread),0.0014,0.00145,100,2.67,0.032,0.008,0.0,5.08,0.187,0.85,0.2,0.0562,0.0,0.00253,0.00415,0.02,0.0,0.0,0.01,0.0,0.44,0.0,0.0,0.00111,0.002,0.0,0.0019,0.0019,0.049,1.33,0.00159,0.0349,0.45,0.01177,1.49,1.77,0.288,0.0119,0.1072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4868,0.04,0.0591,0.0245,0.0,0.0178,0.0,0.0168,0.0,0.0324,0.00777,0.00607,0.01499,0.0,0.0,0.0216,0.0,0.0,0.352
2,Maize (Meal),0.0011,0.00064,100,3.45,0.016,0.003,0.0,0.04,0.378,0.08,0.0,0.06269,0.00145,0.00114,0.0037,0.0,0.0,0.0,0.0,0.0,1.6,0.0,0.0,0.00396,0.0,0.0,0.0057,0.0057,0.022,0.32,0.00343,0.0268,0.96,0.01034,2.96,3.09,0.377,0.02,0.105,0.00409,0.00526,0.00655,0.00232,0.02741,0.0038,0.00236,0.0,0.00383,0.00713,0.00391,0.00202,0.00589,0.01247,0.00443,0.00356,0.00175,0.00301,0.00515,0.7452,0.101,0.008,0.0,0.0,0.0,0.0,0.0,0.0,0.016,0.00335,0.00205,0.00771,0.0,0.0,0.0128,0.0,0.0,0.1211
3,Barley (Beer),0.0011,0.00148,100,0.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.039,0.0011,0.0,0.0,0.9277
4,Oats,0.0016,0.00197,100,3.89,0.069,0.012,0.0,0.02,0.0,0.56,0.0,0.00961,0.01349,0.00139,0.00763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00119,0.0,0.0,0.0,0.0,0.0,0.54,0.00626,0.0472,1.77,0.04916,5.23,4.29,0.0,0.0397,0.1689,0.00881,0.01192,0.01448,0.00408,0.03712,0.00841,0.00405,0.0,0.00694,0.01284,0.00701,0.00312,0.00895,0.00934,0.0075,0.00575,0.00234,0.00573,0.00937,0.6627,0.106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069,0.01217,0.02178,0.02535,0.0,0.0,0.0172,0.0,0.0,0.0822
5,Rice,0.004,0.00134,100,3.57,0.011,0.002,0.0,0.07,0.35,0.95,0.0,0.06733,0.01074,0.00262,0.00115,0.19,0.01,0.0,0.11,0.0,2.2,0.0,0.0,0.00391,0.0,0.0,0.0082,0.0082,0.019,0.21,0.00524,0.0196,1.77,0.01329,4.33,4.27,0.028,0.0596,0.1473,0.00825,0.01136,0.01419,0.00174,0.02565,0.00672,0.00384,0.0,0.00618,0.01018,0.00629,0.00438,0.00721,0.00519,0.00778,0.00469,0.00179,0.00622,0.00858,0.749,0.062,0.025,0.0,0.0,0.0,0.0,0.0,0.0067,0.0108,0.00156,0.00159,0.00676,0.0,0.0,0.0153,0.0,0.0,0.0776


## Price data

In [49]:
price = data[!, "price"];

## Food data

In [50]:
food_products = data[!, "food"];

In [51]:
foods = size(food_products, 1);

### Emissions data

In [52]:
emissions = data[!, "emissions"];

### Nutritional data

In [53]:
calories = data[!, "calories"];

#### Macro nutrients

In [54]:
protein = data[!, "protein"]
total_fat = data[!, "total_fat"]
carbohydrate = data[!, "carbohydrate"]
fiber = data[!, "fiber"]
sugars = data[!, "sugars"]
saturated_fatty_acids = data[!, "saturated_fatty_acids"]
monounsaturated_fatty_acids = data[!, "monounsaturated_fatty_acids"]
polyunsaturated_fatty_acids = data[!, "polyunsaturated_fatty_acids"]
fatty_acids_total_trans = data[!, "fatty_acids_total_trans"];

#### Macro nutrients

In [55]:
vitamin_a = data[!, "vitamin_a"]
vitamin_b6 = data[!, "vitamin_b6"]
vitamin_b12 = data[!, "vitamin_b12"]
vitamin_c = data[!, "vitamin_c"]
vitamin_d = data[!, "vitamin_d"]
vitamin_e = data[!, "vitamin_e"]
vitamin_k = data[!, "vitamin_k"]
thiamin = data[!, "thiamin"]
riboflavin = data[!, "riboflavin"]
niacin = data[!, "niacin"]
folate = data[!, "folate"]
pantothenic_acid = data[!, "pantothenic_acid"]
calcium = data[!, "calcium"]
iron = data[!, "irom"]
magnesium = data[!, "magnesium"]
phosphorous = data[!, "phosphorous"]
potassium = data[!, "potassium"]
sodium = data[!, "sodium"]
zinc = data[!, "zink"]
copper = data[!, "copper"]
manganese = data[!, "manganese"]
selenium = data[!, "selenium"]
choline = data[!, "choline"]
lutein_zeaxanthin = data[!, "lutein_zeaxanthin"]
lucopene = data[!, "lucopene"]
carotene_beta = data[!, "carotene_beta"]
cryptoxanthin_beta = data[!, "cryptoxanthin_beta"]
tocopherol_alpha = data[!, "tocopherol_alpha"];

#### Amino Acids

In [56]:
alanine = data[!, "alanine"]
arginine = data[!, "arginine"]
aspartic_acid = data[!, "aspartic_acid"]
cystine = data[!, "cystine"]
glutamic_acid = data[!, "glutamic_acid"]
glycine = data[!, "glycine"]
histidine = data[!, "histidine"]
hydroxyproline = data[!, "hydroxyproline"]
isoleucine = data[!, "isoleucine"]
leucine = data[!, "leucine"]
lysine = data[!, "lysine"]
methionine = data[!, "methionine"]
phenylalanine = data[!, "phenylalanine"]
proline = data[!, "proline"]
serine = data[!, "serine"]
threonine = data[!, "threonine"]
tryptophan = data[!, "tryptophan"]
tyrosine = data[!, "tyrosine"]
valine = data[!, "valine"];

#### Other nutrients

In [57]:
water = data[!, "water"]
caffeine = data[!, "caffeine"]
theobromine = data[!, "theobromine"]
ash = data[!, "ash"]
alcohol = data[!, "alcohol"];

# Creating model

In [82]:
lambda = 5;

In [83]:
model = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


## Decision variables

In [84]:
@variable(model, x[1:foods] >= 0);

## Objective

In [85]:
# Dual objective function (minimize total carbon emissions and cost)
@objective(model, Min, sum(emissions[i] * x[i] for i in 1:foods) 
    + lambda * sum(price[i] * x[i] for i in 1:foods));

## Contraints

### Macro nutrient constraints

In [86]:
@constraint(model, sum(protein[i] * x[i] for i in 1:foods) >= 50)   # Protein minimum 50g
@constraint(model, sum(total_fat[i] * x[i] for i in 1:foods) <= 78)       # Fat maximum 78g
@constraint(model, sum(carbohydrate[i] * x[i] for i in 1:foods) <= 275); # Carbohydrate maximum 275g

### Micro nutrient constraints

In [87]:
@constraint(model, sum(calcium[i] * x[i] for i in 1:foods) <= 1300) # Calcium maximum 1300mg
@constraint(model, sum(iron[i] * x[i] for i in 1:foods) >= 18)      # Iron minimum 18mg
@constraint(model, sum(magnesium[i] * x[i] for i in 1:foods) <= 420) # Magnesium maximum 420mg
@constraint(model, sum(phosphorous[i] * x[i] for i in 1:foods) <= 1250) # Phosphorus maximum 1250mg
@constraint(model, sum(potassium[i] * x[i] for i in 1:foods) >= 4700) # Potassium minimum 4700mg
@constraint(model, sum(sodium[i] * x[i] for i in 1:foods) <= 2300)  # Sodium maximum 2300mg
@constraint(model, sum(zinc[i] * x[i] for i in 1:foods) <= 11)      # Zinc maximum 11mg
@constraint(model, sum(vitamin_a[i] * x[i] for i in 1:foods) <= 900) # Vitamin A maximum 900mcg RAE
@constraint(model, sum(vitamin_c[i] * x[i] for i in 1:foods) >= 90)  # Vitamin C minimum 90mg
@constraint(model, sum(vitamin_d[i] * x[i] for i in 1:foods) >= 20)  # Vitamin D minimum 20mcg
@constraint(model, sum(vitamin_e[i] * x[i] for i in 1:foods) <= 15)  # Vitamin E maximum 15mg alpha-tocopherol
@constraint(model, sum(vitamin_k[i] * x[i] for i in 1:foods) >= 120) # Vitamin K minimum 120mcg
@constraint(model, sum(riboflavin[i] * x[i] for i in 1:foods) <= 1.3) # Riboflavin maximum 1.3mg
@constraint(model, sum(niacin[i] * x[i] for i in 1:foods) <= 16)    # Niacin maximum 16mg NE
@constraint(model, sum(folate[i] * x[i] for i in 1:foods) <= 400)   # Folate maximum 400mcg DFE
@constraint(model, sum(vitamin_b12[i] * x[i] for i in 1:foods) >= 2.4) # Vitamin B12 minimum 2.4mcg
@constraint(model, sum(vitamin_b6[i] * x[i] for i in 1:foods) <= 1.7) # Vitamin B6 maximum 1.7mg
@constraint(model, sum(choline[i] * x[i] for i in 1:foods) <= 550);  # Choline maximum 550mg

### Other Constraints

In [88]:
@constraint(model, sum(calories[i] * x[i] for i in 1:foods) <= 2500) # Calories maximum at 2500
@constraint(model, sum(calories[i] * x[i] for i in 1:foods) >= 2000); # Calories minimum at 2000
@constraint(model, sum(alcohol[i] * x[i] for i in 1:foods) <= 0); # No Alcohol

## Solving

In [89]:
optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 24 rows, 43 columns and 699 nonzeros
Model fingerprint: 0x353bdd29
Coefficient statistics:
  Matrix range     [1e-04, 5e+01]
  Objective range  [4e-03, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolved: 22 rows, 40 columns, 617 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.849500e+02   0.000000e+00      0s
      18    3.2722900e+01   0.000000e+00   0.000000e+00      0s

Solved in 18 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.272289958e+01

User-callback calls 82, time in user-callback 0.00 sec


In [90]:
# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    # Retrieve optimal values for decision variables
    optimal_diet = value.(x)
    # Retrieve the objective function value
    total_emissions_optimal = objective_value(model)

    # Print the results
    println("Optimal diet: ", optimal_diet)
    println("\nTotal emissions: ", total_emissions_optimal)
else
    println("No optimal solution found. Status: ", termination_status(model))
end

Optimal diet: [0.0, 0.0, 0.0, 71.78536892093202, 0.0, 309.7636346064652, 264.20269378283473, 0.0, 0.0, 36.3258501746699, 8.678974478255807, 0.0, 0.0, 0.0, 165.81501611330654, 41.15869758602767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 223.24452687702754, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.287032095560384, 0.0, 0.0, 548.5856839738543, 0.0, 0.0, 0.0, 0.0]

Total emissions: 32.72289957535572


In [91]:
# Extract the 'Food product' column from the DataFrame
food_product = data[!, "food"]

# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    # Retrieve optimal values for decision variables
    optimal_diet = value.(x)
    # Retrieve the objective function value
    total_emissions = objective_value(model)

    # Print the results
    println("Optimal diet (grams of food):\n")
    for i in 1:length(optimal_diet)
        if optimal_diet[i] > 0
            println(food_product[i], ": ", optimal_diet[i])
        end
    end
    println("\nTotal emissions (kg CO2): ", total_emissions)
else
    println("No optimal solution found. Status: ", termination_status(model))
end

Optimal diet (grams of food):

Oats: 71.78536892093202
Potatoes: 309.7636346064652
Cassava: 264.20269378283473
Beans: 36.3258501746699
Peas: 8.678974478255807
Tofu: 165.81501611330654
Soybean Oil: 41.15869758602767
Berries & Grapes: 223.24452687702754
Lamb & Mutton: 16.287032095560384
Milk: 548.5856839738543

Total emissions (kg CO2): 32.72289957535572


In [92]:
# Calculate total nutrients in the optimized diet

total_nutrients = Dict(
    "calories" => sum(calories[i] * optimal_diet[i] for i in 1:foods),
    "protein" => sum(protein[i] * optimal_diet[i] for i in 1:foods),
    "total_fat" => sum(total_fat[i] * optimal_diet[i] for i in 1:foods),
    "carbohydrate" => sum(carbohydrate[i] * optimal_diet[i] for i in 1:foods),
    "fiber" => sum(fiber[i] * optimal_diet[i] for i in 1:foods),
    "sugars" => sum(sugars[i] * optimal_diet[i] for i in 1:foods),
    "saturated_fatty_acids" => sum(saturated_fatty_acids[i] * optimal_diet[i] for i in 1:foods),
    "monounsaturated_fatty_acids" => sum(monounsaturated_fatty_acids[i] * optimal_diet[i] for i in 1:foods),
    "polyunsaturated_fatty_acids" => sum(polyunsaturated_fatty_acids[i] * optimal_diet[i] for i in 1:foods),
    "fatty_acids_total_trans" => sum(fatty_acids_total_trans[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_a" => sum(vitamin_a[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_b6" => sum(vitamin_b6[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_b12" => sum(vitamin_b12[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_c" => sum(vitamin_c[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_d" => sum(vitamin_d[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_e" => sum(vitamin_e[i] * optimal_diet[i] for i in 1:foods),
    "vitamin_k" => sum(vitamin_k[i] * optimal_diet[i] for i in 1:foods),
    "thiamin" => sum(thiamin[i] * optimal_diet[i] for i in 1:foods),
    "riboflavin" => sum(riboflavin[i] * optimal_diet[i] for i in 1:foods),
    "niacin" => sum(niacin[i] * optimal_diet[i] for i in 1:foods),
    "folate" => sum(folate[i] * optimal_diet[i] for i in 1:foods),
    "pantothenic_acid" => sum(pantothenic_acid[i] * optimal_diet[i] for i in 1:foods),
    "calcium" => sum(calcium[i] * optimal_diet[i] for i in 1:foods),
    "iron" => sum(iron[i] * optimal_diet[i] for i in 1:foods),
    "magnesium" => sum(magnesium[i] * optimal_diet[i] for i in 1:foods),
    "phosphorous" => sum(phosphorous[i] * optimal_diet[i] for i in 1:foods),
    "potassium" => sum(potassium[i] * optimal_diet[i] for i in 1:foods),
    "sodium" => sum(sodium[i] * optimal_diet[i] for i in 1:foods),
    "zinc" => sum(zinc[i] * optimal_diet[i] for i in 1:foods),
    "copper" => sum(copper[i] * optimal_diet[i] for i in 1:foods),
    "manganese" => sum(manganese[i] * optimal_diet[i] for i in 1:foods),
    "selenium" => sum(selenium[i] * optimal_diet[i] for i in 1:foods),
    "choline" => sum(choline[i] * optimal_diet[i] for i in 1:foods),
    "lutein_zeaxanthin" => sum(lutein_zeaxanthin[i] * optimal_diet[i] for i in 1:foods),
    "lucopene" => sum(lucopene[i] * optimal_diet[i] for i in 1:foods),
    "carotene_beta" => sum(carotene_beta[i] * optimal_diet[i] for i in 1:foods),
    "cryptoxanthin_beta" => sum(cryptoxanthin_beta[i] * optimal_diet[i] for i in 1:foods),
    "tocopherol_alpha" => sum(tocopherol_alpha[i] * optimal_diet[i] for i in 1:foods),
    "alanine" => sum(alanine[i] * optimal_diet[i] for i in 1:foods),
    "arginine" => sum(arginine[i] * optimal_diet[i] for i in 1:foods),
    "aspartic_acid" => sum(aspartic_acid[i] * optimal_diet[i] for i in 1:foods),
    "cystine" => sum(cystine[i] * optimal_diet[i] for i in 1:foods),
    "glutamic_acid" => sum(glutamic_acid[i] * optimal_diet[i] for i in 1:foods),
    "glycine" => sum(glycine[i] * optimal_diet[i] for i in 1:foods),
    "histidine" => sum(histidine[i] * optimal_diet[i] for i in 1:foods),
    "hydroxyproline" => sum(hydroxyproline[i] * optimal_diet[i] for i in 1:foods),
    "isoleucine" => sum(isoleucine[i] * optimal_diet[i] for i in 1:foods),
    "leucine" => sum(leucine[i] * optimal_diet[i] for i in 1:foods),
    "lysine" => sum(lysine[i] * optimal_diet[i] for i in 1:foods),
    "methionine" => sum(methionine[i] * optimal_diet[i] for i in 1:foods),
    "phenylalanine" => sum(phenylalanine[i] * optimal_diet[i] for i in 1:foods),
    "proline" => sum(proline[i] * optimal_diet[i] for i in 1:foods),
    "serine" => sum(serine[i] * optimal_diet[i] for i in 1:foods),
    "threonine" => sum(threonine[i] * optimal_diet[i] for i in 1:foods),
    "tryptophan" => sum(tryptophan[i] * optimal_diet[i] for i in 1:foods),
    "tyrosine" => sum(tyrosine[i] * optimal_diet[i] for i in 1:foods),
    "valine" => sum(valine[i] * optimal_diet[i] for i in 1:foods)
)

# Print the total nutrients rounded to three decimal places
println("Total nutrients in the optimized diet:\n")
for (nutrient, total) in total_nutrients
    println(nutrient, ": ", round(total, digits=3))
end

Total nutrients in the optimized diet:

calories: 2000.0
riboflavin: 1.177
valine: 2.539
total_fat: 78.0
vitamin_b6: 1.7
phosphorous: 1250.0
tocopherol_alpha: 4.842
glycine: 1.584
histidine: 1.195
fatty_acids_total_trans: 92.6
serine: 2.211
sodium: 163.027
carotene_beta: 230.22
cryptoxanthin_beta: 0.0
thiamin: 1.456
lutein_zeaxanthin: 375.714
aspartic_acid: 3.931
proline: 3.024
vitamin_b12: 2.4
vitamin_d: 280.104
vitamin_a: 900.0
arginine: 2.609
niacin: 8.751
protein: 72.481
magnesium: 363.086
copper: 2.529
choline: 348.672
cystine: 0.667
tryptophan: 0.603
sugars: 67.157
carbohydrate: 275.0
pantothenic_acid: 4.31
iron: 20.123
vitamin_e: 4.842
folate: 400.0
leucine: 3.629
hydroxyproline: 0.0
monounsaturated_fatty_acids: 14.847
vitamin_c: 111.19
zinc: 8.703
tyrosine: 1.604
vitamin_k: 120.0
methionine: 0.932
glutamic_acid: 8.81
threonine: 1.777
fiber: 29.736
calcium: 1053.157
isoleucine: 2.14
selenium: 28.776
saturated_fatty_acids: 21.988
polyunsaturated_fatty_acids: 26.216
alanine: 1.958

In [93]:
x = JuMP.value.(x)
total_price = sum(x[i] * price[i] for i in 1:foods)

5.857445703108965